# 2.- MODEL TRAINING<br>
In this notebook, the COST FUCNTION for the ML is definied using tensorflow, and MATRIX FACTORIZATION parameters are computed using GRADIENT DESCENT 

## Libraries

In [ ]:
##########################################################################################################################

In [1]:
import sys
import numpy
import numpy        as     np
import pandas       as     pd
import tensorflow   as     tf
from   tensorflow   import keras

numpy.set_printoptions(threshold=20)

## Import Information

In [ ]:
###########################################################################################################################

In [2]:
# It may take some time

normalized_UtilityMatrix   =  pd.read_csv  ('normalized_UtilityMatrix .csv') 
UtilityMatrix_Reviewed     =  pd.read_csv  ('UtilityMatrix_Reviewed.csv'   )
my_index_reviewed          =  pd.read_csv  ('my_index_reviewed.csv'        )
mean_recipe                =  pd.read_csv  ('mean_recipe.csv'              )

In [ ]:
###########################################################################################################################

## Define Cost Function<BR>

In [3]:
def CostFunction(X, W, b, UtilityMatrix, UtilityMatrix_Reviewed, lambda_):

    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - UtilityMatrix)*UtilityMatrix_Reviewed
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    
    return J

In [4]:
normalized_UtilityMatrix   =  normalized_UtilityMatrix.transpose()
UtilityMatrix_Reviewed     =  UtilityMatrix_Reviewed.transpose()

In [5]:
my_index_reviewed          =  my_index_reviewed['0'].tolist()
normalized_UtilityMatrix   =  normalized_UtilityMatrix.to_numpy()
UtilityMatrix_Reviewed     =  UtilityMatrix_Reviewed.to_numpy()
mean_recipe                =  mean_recipe.to_numpy()

In [6]:
#  We set the number of features to 100 

num_recipes, num_users = normalized_UtilityMatrix.shape
num_features           = 100


# for consistent results

tf.random.set_seed(1234)  

# Set Initial Parameters (W, X), use tf.Variable to track these variables

W = tf.Variable(tf.random.normal ( (  num_users,  num_features),  dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal ( (num_recipes,  num_features),  dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal ( (          1,     num_users),  dtype=tf.float64),  name='b')

# Instantiate an optimizer.

optimizer = keras.optimizers.Adam(learning_rate=1e-1)

## MATRIX FACTORIZATION Parameters Calculation <br>
-    Code adaptation from https://aman.ai/recsys/candidate-gen/
- Computation takes around 20 minutes in my computer

In [7]:
iterations = 120     
lambda_    = 1

for iter in range(iterations):
    
    # Use TensorFlow’s GradientTape
    # to record the operations used to compute the cost 
    
    with tf.GradientTape() as tape:

        # Compute the cost (forward pass included in cost)
        
        loss = CostFunction(X, W, b, normalized_UtilityMatrix, UtilityMatrix_Reviewed, lambda_)

    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss
    
    grads = tape.gradient( loss, [X,W,b] )

    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    
    optimizer.apply_gradients( zip(grads, [X,W,b]) )

    # Print the training loss every 20 iterations 
    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {loss:0.1f}")

Training loss at iteration 0: 22707855.4
Training loss at iteration 20: 1295055.1
Training loss at iteration 40: 574151.5
Training loss at iteration 60: 290785.1
Training loss at iteration 80: 161837.6
Training loss at iteration 100: 97321.2


## Matrix Factors<br>
- **W** Factors corresponding to Recipes
- **X & b** Factors corresponding to Users

In [8]:
W = np.transpose(W.numpy())
X = X.numpy()
b = b.numpy()

## Matrix Factors only for the USER are Extracted

In [9]:
# Extract from Matrix Factorization, the factors to predict User's Reviews

W_myReview = W[:,0]
b_myReview = b[:,0].reshape(-1,1)

## Save Files 

In [10]:
##########################################################################################################################

In [11]:
# X constains all factors corresponding to the Recipe Matrix

X = pd.DataFrame(X)
X.to_csv('X.csv',index= False)

# W_myRevies contains the factors plus the bias b_myReview corresponding to The User Matrix

W_myReview = pd.DataFrame(W_myReview)
W_myReview.to_csv('W_myReview.csv',index= False)

b_myReview = pd.DataFrame(b_myReview)
b_myReview.to_csv('b_myReview.csv',index= False)

In [ ]:
##########################################################################################################################

In [ ]:
##########################################################################################################################

In [ ]:
##########################################################################################################################

# Continue with PREDICTIONS notebook